In [ ]:
import warnings; warnings.filterwarnings('ignore')
import numpy as np,pandas as pd,pylab as pl
import glob
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical
from keras.models import Sequential,Model
from keras.optimizers import SGD,RMSprop,Adam,Nadam
from keras.callbacks import ModelCheckpoint,EarlyStopping
from keras.callbacks import ReduceLROnPlateau
from keras.layers import Dense,Dropout,LSTM
from keras.layers import Activation,Flatten,Input,BatchNormalization
from keras.layers import Conv1D,MaxPooling1D,Conv2D,MaxPooling2D
from keras.layers import GlobalAveragePooling2D,GlobalMaxPooling2D

In [ ]:
path='../input/svhn-preproccessed-fragments/'
fw='weights.housenumbers.hdf5'
glob.glob(path+'*')

<h1 class='font-effect-3d' style='font-family:Akronim; color:#ff55ee'> Data Loading & Preprocessing</h1>

In [ ]:
import cv2
import numpy as np
train_images=pd.read_csv(path+'train_images.csv')
train_labels=pd.read_csv(path+'train_labels.csv')
test_images=pd.read_csv(path+'test_images.csv')
test_labels=pd.read_csv(path+'test_labels.csv')
extra_images=pd.read_csv(path+'extra_images.csv')
extra_labels=pd.read_csv(path+'extra_labels.csv')

train_images=train_images.iloc[:,1:]\
.as_matrix().astype('float32').reshape(-1,32,32,1)
train_labels=train_labels.iloc[:,1:]\
.as_matrix().astype('int16')
test_images=test_images.iloc[:,1:]\
.as_matrix().astype('float32').reshape(-1,32,32,1)
test_labels=test_labels.iloc[:,1:]\
.as_matrix().astype('int16')
extra_images=extra_images.iloc[:,1:]\
.as_matrix().astype('float32').reshape(-1,32,32,1)
extra_labels=extra_labels.iloc[:,1:]\
.as_matrix().astype('int16')


n=np.random.randint(1,2000,1)[0]
print('Label: ',train_labels[n])
#print(ctrain_labels[n])
pl.imshow(train_images[n].reshape(32,32),
          cmap=pl.cm.bone);

'''
train_images_t=train_images.iloc[:,1:]\
.as_matrix().astype('float32').reshape(-1,32,32,1)
train_labels=train_labels.iloc[:,1:]\
.as_matrix().astype('int16')
test_images_t=test_images.iloc[:,1:]\
.as_matrix().astype('float32').reshape(-1,32,32,1)
test_labels=test_labels.iloc[:,1:]\
.as_matrix().astype('int16')
extra_images_t=extra_images.iloc[:,1:]\
.as_matrix().astype('float32').reshape(-1,32,32,1)
extra_labels=extra_labels.iloc[:,1:]\
.as_matrix().astype('int16')
train_images= []
for img in train_images_t:
    img= cv2.resize(img, (50, 50), interpolation= cv2.INTER_CUBIC)
    train_images.append(img)
    
train_images= np.asarray(train_images)

test_images= []
for img in test_images_t:
    img= cv2.resize(img, (50, 50), interpolation= cv2.INTER_CUBIC)
    test_images.append(img)

train_images= np.asarray(test_images)

extra_images= []
for img in extra_images_t:
    img= cv2.resize(img, (50, 50), interpolation= cv2.INTER_CUBIC)
    extra_images.append(img)
extra_images= np.asarray(extra_images)

n=np.random.randint(1,2000,1)[0]
print('Label: ',train_labels[n])
#print(ctrain_labels[n])
pl.imshow(train_images[n].reshape(50,50),
          cmap=pl.cm.bone);
'''

In [ ]:
n=np.random.randint(1,2000,1)[0]
print('Label: ',train_labels[n])
#print(ctrain_labels[n])
pl.imshow(train_images[n].reshape(32,32),
          cmap=pl.cm.bone);

Add in. Take only the numbers 3 and above

In [ ]:
import random
print(train_labels.shape)
print(train_images.shape)
print(test_labels.shape)
print(test_images.shape)
print(extra_labels.shape)
print(extra_images.shape)

def reducesize(orglabel,orgimgs):
    first = True

    newlabel= []
    newimgs= []
    
    for x, label in enumerate(orglabel):
        idx= 0
        cnt= 0
        #if x >= 7000:
            #return newlabel, newimgs
        for chk in label:
            if chk == 10 and idx < 2:
                cnt = cnt+1 #check if the first two numbers are null
            idx=idx+1
        if cnt >= 2:
            label= np.delete(label, [0, 1])
            label= np.expand_dims(label, axis= 0)
            
            Ydim,Xdim,chan= orgimgs[x].shape
            orgX= Xdim
            orgY= Ydim
            num1= random.randint(round(Xdim*0.6), round(Xdim*0.9))
            #print("Shape: "+ str(orgimgs[x].shape))
            Xcolour= random.randint(60, 128) #color of padding
            x_add= np.full((Ydim,num1), Xcolour/255.0,dtype= float)
            x_add= np.expand_dims(x_add, axis= 2)
            #print("Shape: "+ str(x_add.shape))
            img= np.append(x_add,orgimgs[x],axis= 1)
            img= np.append(img,x_add,axis= 1)
            #padding Y
            Ydim, Xdim, chan= img.shape
            num2= random.randint(round(Ydim*0.6), round(Ydim*0.9))
            Ycolour= random.randint(10, 100)
            y_add= np.full((num2,Xdim), Ycolour/255.0,dtype=float)
            y_add= np.expand_dims(y_add, axis= 2)
            img= np.append(y_add, img,axis= 0)
            img= np.append(img, y_add,axis= 0)
            img= cv2.resize(img, (32,32))
            img= np.expand_dims(img, axis= 0)
            img= np.expand_dims(img, axis= 3)

            #data aug end
            
            if first:
                newlabel= label
                newimgs= img
                #print(img.shape)
                #print(label.shape)
                first = False
                print("Shape aft padding: "+ str(img.shape))
                #print(label)
            else:
                newlabel= np.append(newlabel, label, axis= 0)
                newimgs= np.append(newimgs, img, axis= 0)
                
                #print(label)
    return newlabel, newimgs

newtest_labels, newtest_imgs= reducesize(orglabel= test_labels, orgimgs= test_images)
print("NewTest_labels"+str(newtest_labels.shape))
print("NewTest_imgs"+str(newtest_imgs.shape))
newtrain_labels, newtrain_imgs= reducesize(orglabel= train_labels, orgimgs= train_images)
print("NewTrain_labels"+str(newtrain_labels.shape))
print(newtrain_imgs.shape)
newextra_labels, newextra_imgs= reducesize(orglabel= extra_labels, orgimgs= extra_images)
print(newextra_labels.shape)
print(newextra_imgs.shape)

In [ ]:
print(newtest_imgs)

In [ ]:
n=np.random.randint(1,len(newtrain_labels),1)[0]
print('Label: ',newtrain_labels[n])
#print(ctrain_labels3[n])
pl.imshow(newtrain_imgs[n].reshape(32,32),
          cmap=pl.cm.bone);

In [ ]:
ctrain_labels=to_categorical(newtrain_labels,num_classes=11)\
.astype('int16')
ctest_labels=to_categorical(newtest_labels,num_classes=11)\
.astype('int16')
cextra_labels=to_categorical(newextra_labels,num_classes=11)\
.astype('int16')

In [ ]:
train_images= newtrain_imgs
test_images= newtest_imgs
extra_images= newextra_imgs

In [ ]:
n=np.random.randint(1,len(train_labels),1)[0]
print('Label\n',ctrain_labels[n])
#print(ctrain_labels[n])
pl.imshow(train_images[n].reshape(32,32),
          cmap=pl.cm.bone);

End of added in stuff

In [ ]:
Nums= 3

X=np.concatenate((train_images,
                        test_images),axis=0)
X=np.concatenate((X,extra_images),axis=0)
y=np.concatenate((ctrain_labels,
                  ctest_labels),axis=0)
y=np.concatenate((y,cextra_labels),axis=0)
def tts(X,y): 
    x_train,x_test,y_train,y_test=\
    train_test_split(X,y,test_size=.2,random_state=1)
    n=int(len(x_test)/2)
    x_valid,y_valid=x_test[:n],y_test[:n]
    x_test,y_test=x_test[n:],y_test[n:]
    return x_train,x_valid,x_test,y_train,y_valid,y_test
x_train,x_valid,x_test,\
y_train,y_valid,y_test=tts(X,y)
y_train_list=[y_train[:,i] for i in range(Nums)]
y_test_list=[y_test[:,i] for i in range(Nums)]
y_valid_list=[y_valid[:,i] for i in range(Nums)]
for el in [x_train,x_valid,x_test,
           y_train,y_valid,y_test]:
    print(el.shape)

In [ ]:
print(y_train.shape)

In [ ]:
print('Label\n',y_train_list[0][0])
print('Label\n',y_train_list[0][1])
print('Label\n',y_train_list[0][2])
#print(ctrain_labels[n])
pl.imshow(x_train[0].reshape(32,32),
          cmap=pl.cm.bone);

<h1 class='font-effect-3d' style='font-family:Akronim; color:#ff55ee'>Build the Model</h1>

In [ ]:
def cnn_model():    
    model_input=Input(shape=(32,32,1))
    x=BatchNormalization()(model_input)        
    x=Conv2D(32,(3,3),activation='relu',
             padding='same')(model_input)
    x=MaxPooling2D(pool_size=(2,2))(x)     
    x=Conv2D(32,(3,3),activation='relu')(x)
    x=MaxPooling2D(pool_size=(2,2))(x)    
    x=Dropout(.25)(x)    
    x=Conv2D(64,(3,3),activation='relu')(x)       
    x=Conv2D(64,(3,3),activation='relu')(x)    
    x=Dropout(.25)(x)    
    x=Conv2D(196,(3,3),activation='relu')(x)    
    x=Dropout(.25)(x)              
    x=Flatten()(x)    
    x=Dense(512,activation='relu')(x)    
    x=Dropout(.5)(x)    
    y=[Dense(11,activation='softmax')(x)
       for i in range(Nums)]    
    model=Model(input=model_input,output=y)
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',metrics=['accuracy'])
    return model
cnn_model=cnn_model()

In [ ]:
checkpointer=ModelCheckpoint(filepath=fw,verbose=2,
                             save_best_only=True)
lr_reduction=ReduceLROnPlateau(monitor='val_loss',patience=10,
                               verbose=2,factor=.75)
estopping=EarlyStopping(monitor='val_loss',patience=16,verbose=2)
history=cnn_model.fit(x_train,y_train_list,
                      validation_data=(x_valid,y_valid_list), 
                      epochs=250,batch_size=128,verbose=2, 
                      callbacks=[checkpointer,lr_reduction,estopping])

In [ ]:
cnn_model.load_weights(fw)
cnn_scores=cnn_model.evaluate(x_test,y_test_list,verbose=0)
print("CNN. Scores: \n" ,(cnn_scores))

In [ ]:
cnn_model.save('./numfinder')